In [1]:
import pinocchio as pin
import pinocchio.casadi as cpin
import casadi as cs
from pinocchio.robot_wrapper import RobotWrapper
import sys, os
import numpy as np
from enum import Enum
from pathlib import Path

In [5]:
urdf_path = '../models/unitree_go1/go1.urdf'
mesh_dir = "../models/unitree_go1"
model = RobotWrapper.BuildFromURDF(urdf_path, mesh_dir, root_joint = pin.JointModelFreeFlyer()).model
data = model.createData()

In [18]:
nq, nv = model.nq, model.nv
q = pin.neutral(model)
v = np.zeros(nv)

pin.forwardKinematics(model, data, q, v)
pin.updateFramePlacements(model, data)

pin.getFramePosition(model, data, frame_id, pin.ReferenceFrame.WORLD)
